In [88]:
import os
import numpy as np


                
def toQuery(rootdir,outtxt,folderpos=0):
    
    #
    
    outfile = open(outtxt,'w')
    
    for subdir, dirs, files in os.walk(rootdir):
        for i in range(len(files)):
            if i == folderpos:
                tempO = open(os.path.join(subdir, files[i]))
                contents = tempO.read()
                for i in range(len(contents)):
                    outfile.write(contents[i])

                    
                    
def toDb(rootdir,outfna):
    
    #
    
    outfile = open(outfna,'w')
    
    for subdir, dirs, files in os.walk(rootdir):
        for i in range(len(files)):
            tempO = open(os.path.join(subdir, files[i]))
            contents = tempO.read()
            for i in range(len(contents)):
                outfile.write(contents[i])

                

def blastnCount(inOut):
    
    #
    
    blastnOut = open(inOut)
    
    seqcount = 0 #number of found matches
    totcount = 0 #number of sequences found in query 
    
    line = blastnOut.readline()
    while line:
        if (line == 'Sequences producing significant alignments:                          (Bits)  Value\n'):
            seqcount = seqcount + 1
            totcount = totcount + 1
        if (line == '***** No hits found *****\n'):
            totcount = totcount + 1
        line = blastnOut.readline()
    blastnOut.close()
    print(seqcount)
    print(totcount)



def onlyHits(infile,outfile):
    
    #
    
    fullBlastn = open(infile)
    onlyHits = open(outfile,'w')
    
    newblock = []
    hitcheck = 0
    
    line = fullBlastn.readline()
    while line:
        if((line[0] == 'Q') and (line[5] == '=')):
            if(newblock and hitcheck):
                for i in range(len(newblock)):
                    onlyHits.write(newblock[i])
            newblock = []
            hitcheck = 0 
        if(line == 'Sequences producing significant alignments:                          (Bits)  Value\n'):
            hitcheck = 1
        newblock.append(line)
        line = fullBlastn.readline()
    if(hitcheck):
        for i in range(len(newblock)):
            onlyHits.write(newblock[i])
        newblock = []
        hitcheck = 0
    fullBlastn.close()
            

def toFasta(infile,outfile):
    
    #
    
    onlyHits = open(infile)
    fasForm = open(outfile,'w')
    
    newline = ''
    scorecheck = 0
    
    line = onlyHits.readline()
    while line:
        if(line[1:6] == "Score"):
            scorecheck = scorecheck + 1
        elif((line[0]) == 'Q') and (line[5] == '='):
            scorecheck = 0
            if(newline):
                fasForm.write(newline)
                fasForm.write('\n')
                newline = ''
            fasForm.write('>')
            check1 = 0
            line = line[7:]
            while(check1 == 0):
                for i in range(len(line)):
                    if(line[i] != '\n'):
                        fasForm.write(line[i])
                    else:
                        fasForm.write(' ')
                line = onlyHits.readline()
                if(len(line)<=1):
                    check1 = 1
            fasForm.write('\n')
        elif((line[0] == 'Q') and (line[5] != '=')):
            if(scorecheck < 2):
                for i in range(len(line)):
                    if((line[i] == 'A' or line[i] == 'T') or (line[i] == 'C' or line[i] == 'G')):
                        newline = newline + line[i]
        line = onlyHits.readline()
    fasForm.write(newline)
    onlyHits.close()


def lengthAve(infile,lowcut=0,highcut=1):
    
    #takes in onlyHits to fasta file. lowcut,hightcut are bottom and top percentile cutoffs for counting average length
    #default parameters will average all sequences. 
    
    inf = open(infile)
    
    alllengths = []
    line = inf.readline()
    while line:
        if(line[0] != '>'):
            curcount = 0
            line = line[1:-1]
            alllengths.append(len(line))
        line = inf.readline()
    inf.close()
    
    alllengths.sort()
    numcount = 0
    lencount = 0
    for i in range(int(round(lowcut*len(alllengths))),int(round(highcut*len(alllengths)))):
        numcount = numcount + 1
        lencount = lencount + alllengths[i]
        
    print(lencount/numcount)
    print(alllengths)
    
    
def removeMarks(infile,outfile):
    
    #
    
    mothurAlign = open(infile)
    out = open(outfile,'w')
    
    line = mothurAlign.readline()
    while line:
        for i in range(len(line)):
            if((line[i] != '.') and (line[i] != '-')):
                out.write(line[i])
        line = mothurAlign.readline()
    mothurAlign.close()
    
    
def originalTags(infile,template,outfile):
    
    #
    
    inf = open(infile)
    tem = open(template)
    outf = open(outfile,'w')
    
    line = inf.readline()
    line = tem.readline()
    check = 1
    while(line):
        outf.write(line)
        check = check * -1
        if(check == -1):
            line = tem.readline()
            line = inf.readline()
        else:
            line = inf.readline()
            line = tem.readline()
        
    inf.close()
    tem.close()
    
def phylipFormat(infile,outfile,seqnum):
    
    #seqnum = '259\n' if there are 259 sequences
    
    megaCSV = open(infile)
    mothurPhylip = open(outfile,'w')
    
    mothurPhylip.write('259\n')
    
    line = megaCSV.readline()
    while line:
        for i in range(len(line)):
            if(line[i] == ','):
                mothurPhylip.write('\t')
            else:
                mothurPhylip.write(line[i])
        line = megaCSV.readline()
        if(line[0] == '\n'):
            line = ''
    megaCSV.close()

I'm leaving the previous processing of glnII from the other notebook to use as a reference

In [13]:
#glnII
toDb('glnIIrefs','glnIIref.fna')

In [18]:
blastnCount('glnIIout.txt')

218
15248


In [16]:
onlyHits('glnIIout.txt','glnIIoutOnlyHits.txt')

In [17]:
toFasta('glnIIoutOnlyHits.txt','glnIIout.fasta')

In [38]:
removeMarks('glnIIgpo.align','glnIIgpoflipped.fasta')

In [39]:
originalTags('glnIIgpoflipped.fasta','glnIIgpo.fasta','glnIIgpoformflip.fasta')

In [60]:
lengthAve('redo/16srRNAout.fasta',lowcut = 0.3,highcut = 0.95)


1481.3034825870648
[40, 45, 52, 52, 63, 65, 65, 71, 85, 87, 99, 108, 109, 116, 126, 130, 130, 134, 160, 167, 177, 220, 231, 242, 259, 259, 264, 266, 280, 309, 346, 421, 456, 466, 481, 562, 616, 663, 690, 721, 747, 929, 940, 964, 965, 994, 1120, 1132, 1151, 1160, 1253, 1275, 1283, 1299, 1300, 1310, 1354, 1370, 1381, 1386, 1386, 1386, 1400, 1405, 1405, 1405, 1405, 1459, 1472, 1472, 1472, 1472, 1472, 1472, 1472, 1472, 1472, 1472, 1472, 1472, 1472, 1472, 1472, 1472, 1472, 1472, 1472, 1474, 1474, 1474, 1476, 1476, 1478, 1478, 1479, 1480, 1480, 1480, 1480, 1480, 1480, 1480, 1480, 1480, 1480, 1480, 1480, 1480, 1480, 1480, 1480, 1480, 1480, 1480, 1480, 1480, 1480, 1480, 1480, 1480, 1480, 1480, 1480, 1480, 1480, 1480, 1480, 1480, 1480, 1480, 1480, 1480, 1480, 1480, 1480, 1480, 1480, 1480, 1480, 1480, 1480, 1480, 1480, 1480, 1480, 1480, 1480, 1480, 1480, 1480, 1480, 1480, 1480, 1480, 1480, 1480, 1480, 1480, 1480, 1480, 1480, 1480, 1480, 1480, 1480, 1480, 1480, 1480, 1480, 1480, 1480, 1480, 1480,

All below is new

In [6]:
blastnCount("plasmid_blast/plasmidnifHout.txt")
blastnCount("plasmid_blast/plasmidnodCout.txt")

54
265
51
265


In [7]:
onlyHits("plasmid_blast/plasmidnifHout.txt","plasmid_blast/PnifHoutOH.txt")
onlyHits("plasmid_blast/plasmidnodCout.txt","plasmid_blast/PnodCoutOH.txt")

In [9]:
toFasta("plasmid_blast/PnodCoutOH.txt","plasmid_blast/PnodC.fasta")
toFasta("plasmid_blast/PnifHoutOH.txt","plasmid_blast/PnifH.fasta")

In [9]:
#run to make table base for functional gene annotation

import csv
with open('csvs/fulluniquetable.csv','r') as f:
    reader = csv.reader(f)
    FUT = list(reader)
    
genpres = []

for i in range(len(FUT)):
    if(FUT[i][5] != '-'):
        genpres.append([FUT[i][0],FUT[i][4],FUT[i][5]])

In [10]:
#takes fasta from above old functions, base table from cell above, name of the gene to add column name
#puts length of match instead of a binary "1" when a match exists 


import os

def addpresLength(fasta,table,genename):
    if(genename in table[0]):
        return table
    
    preslist = []
    lenlist = []
    
    fas = open(fasta)
    line = fas.readline()
    while(line):
        if(line[0] == '>'):
            curRS = ""
            pos = 1
            while(line[pos] != ' '):
                curRS += line[pos]
                pos += 1
            preslist.append(curRS)
        else:
            lenlist.append(len(line) - 2)
            
        line = fas.readline()
        
    table[0].append(genename)
    
    for i in range(1, len(table)):
        notfound = True
        for j in range(len(preslist)):
            if(table[i][2] == preslist[j]):
                table[i].append(lenlist[j])
                notfound = False
        if(notfound):
            table[i].append(0)
        
    
    return table
                
            

In [105]:
#takes the onlyHits output to create a fasta like score file of the highest score for plasmid
#plasmids without matches will not be included in the score file


def toScore(inf,outf):
    
    onlyHits = open(inf)
    scoretext = open(outf,'w')
    
    scoretrack = []
    
    line = onlyHits.readline()
    while(line):
        if(line[0:7] == "Query= "):
            if(len(scoretrack) > 0):
                highscore = 0
                for score in scoretrack:
                    if(int(round(float(score))) > highscore):
                        highscore = int(round(float(score)))
                scoretext.write(str(highscore))
                scoretext.write('\n')
            
            scoretext.write('>')
            line = line[7:]
            while(len(line) > 1):
                for i in range(len(line)):
                    if(line[i] != '\n'):
                        scoretext.write(line[i])
                    else:
                        scoretext.write(' ')
                line = onlyHits.readline()
            scoretext.write('\n')
            
            scoretrack = []
        
        if(line[1:9] == "Score = "):
            curscore = ""
            curpos = 9
            while(line[curpos] != ' '):
                curscore += line[curpos]
                curpos += 1
            scoretrack.append(curscore)
            
        line = onlyHits.readline()
        
    highscore = 0
    for score in scoretrack:
        if(int(score) > highscore):
            highscore = int(score)
    scoretext.write(str(highscore))

    

In [11]:
#tkaes in the score file from above, does same thing as "addpresLength" but with scores given by the blast output


import os

def addpresScore(scoretext,table,genename):
    if(genename in table[0]):
        return table
    
    preslist = []
    scorelist = []
    
    scores = open(scoretext)
    line = scores.readline()
    while(line):
        if(line[0] == '>'):
            curRS = ""
            pos = 1
            while(line[pos] != ' '):
                curRS += line[pos]
                pos += 1
            preslist.append(curRS)
        else:
            if(line[-1:] == '\n'):
                line = line[:-1]
            scorelist.append(int(line))
            
        line = scores.readline()
        
    table[0].append(genename)
    
    for i in range(1, len(table)):
        notfound = True
        for j in range(len(preslist)):
            if(table[i][2] == preslist[j]):
                table[i].append(scorelist[j])
                notfound = False
        if(notfound):
            table[i].append(0)
        
    
    return table
                
            

In [ ]:
toScore("plasmid_blast/PnifHoutOH.txt","plasmid_blast/PnifHscore.txt")

In [ ]:
toScore("plasmid_blast/PnodCoutOH.txt","plasmid_blast/PnodCscore.txt")

In [13]:
#rerun this when creating genpres 


genpres = addpresScore("plasmid_blast/PnifHscore.txt",genpres,"nifH")
genpres = addpresScore("plasmid_blast/PnodCscore.txt",genpres,"nodC")

Additional genes start here

In [22]:
#fixA
#this runthrough should be more organized than above and should be a good template for further additions

blastnCount("plasmid_blast/fixAout.txt")


40
265


In [87]:
onlyHits("plasmid_blast/fixAout.txt","plasmid_blast/fixAoutOH.txt")

In [25]:
toScore("plasmid_blast/fixAoutOH.txt","plasmid_blast/fixAscore.txt")

In [26]:
genpres = addpresScore("plasmid_blast/fixAscore.txt",genpres,"fixA")

In [83]:
#acpP

blastnCount("plasmid_blast/acpPout.txt")

0
265


In [95]:
#actP

blastnCount("plasmid_blast/actPout.txt")

55
265


In [106]:
onlyHits("plasmid_blast/actPout.txt","plasmid_blast/actPoutOH.txt")

In [108]:
toScore("plasmid_blast/actPoutOH.txt","plasmid_blast/actPscore.txt")

In [110]:
genpres = addpresScore("plasmid_blast/actPscore.txt",genpres,"actP")

In [112]:
#amtB

blastnCount("plasmid_blast/amtBout.txt")

0
265


In [113]:
#hemA

blastnCount("plasmid_blast/hemAout.txt")

38
265


In [114]:
onlyHits("plasmid_blast/hemAout.txt","plasmid_blast/hemAoutOH.txt")

In [115]:
toScore("plasmid_blast/hemAoutOH.txt","plasmid_blast/hemAscore.txt")

In [117]:
genpres = addpresScore("plasmid_blast/hemAscore.txt",genpres,"hemA")

In [118]:
#modB

blastnCount("plasmid_blast/modBout.txt")

0
265


Saving table as csv below here. Should probably rename the file with each save to not overwrite

I probably should also write functions for these.

In [123]:
#LIST TO TEXT

import os

funcplas = open("plasmid_blast/funcpres2.txt",'w')

for i in range(len(genpres)):
    for j in range(len(genpres[i])):
        funcplas.write(str(genpres[i][j]))
        if(j < (len(genpres[i]) - 1)):
            funcplas.write('\t')
    funcplas.write('\n')

In [124]:
#TEXT TO CSV 

import csv

txt_file = r"plasmid_blast/funcpres2.txt"
csv_file = r"plasmid_blast/funcpres2.csv"

with open(txt_file, "r") as in_text:
    in_reader = csv.reader(in_text, delimiter = '\t')
    with open(csv_file, "w") as out_csv:
        out_writer = csv.writer(out_csv)
        for row in in_reader:
            out_writer.writerow(row)

Adding symbiotic gene information to the 2 inf tables

In [122]:
len(genpres)

266

In [129]:
symbiotic = []

for i in range(1,len(genpres)):
    curcount = 0
    for j in range(3):
        if(genpres[i][j+3] != 0):
            curcount += 1
    if(curcount >= 2):
        symbiotic.append([genpres[i][0],genpres[i][2]])

In [132]:
symbiotic[0]

['Rhizobium etli', 'NZ_CP020910.1']

In [143]:
#CSV TO LIST
import csv
with open('csvs/plasmidinf.csv','r') as f:
    reader = csv.reader(f)
    plasmidinf = list(reader)

In [149]:
plasmidinf[0].insert(8,"Symbiotic")

for i in range(1,len(plasmidinf)):
    found = False
    for j in range(len(symbiotic)):
        if(plasmidinf[i][6] == symbiotic[j][1]):
            found = True
            symbiotic[j].append(plasmidinf[i][0])
            symbiotic[j].append(plasmidinf[i][5])
            continue
    if(found):
        plasmidinf[i].insert(8,"Yes")
    else:
        plasmidinf[i].insert(8,'')
    

In [142]:
import csv
with open('csvs/organisminf.csv','r') as f:
    reader = csv.reader(f)
    organisminf = list(reader)

In [155]:
organisminf[0].insert(2,"Symbiotic plasmid")

for i in range(1,len(organisminf)):
    found = False
    foundind = 999
    for j in range(len(symbiotic)):
        if(symbiotic[j][2] == organisminf[i][0]):
            found = True
            foundind = j 
            continue
    if(found):
        organisminf[i].insert(2,symbiotic[foundind][3])
    else:
        organisminf[i].insert(2,'')

In [161]:
#LIST TO TEXT

import os

funcplas = open("plasmid_blast/organisminf.txt",'w')

for i in range(len(organisminf)):
    for j in range(len(organisminf[i])):
        funcplas.write(str(organisminf[i][j]))
        if(j < (len(organisminf[i]) - 1)):
            funcplas.write('\t')
    funcplas.write('\n')
funcplas.close()

funcplas = open("plasmid_blast/plasmidinf.txt",'w')

for i in range(len(plasmidinf)):
    for j in range(len(plasmidinf[i])):
        funcplas.write(str(plasmidinf[i][j]))
        if(j < (len(plasmidinf[i]) - 1)):
            funcplas.write('\t')
    funcplas.write('\n')
funcplas.close()

In [162]:
#TEXT TO CSV 

import csv

txt_file = r"plasmid_blast/organisminf.txt"
csv_file = r"plasmid_blast/organisminf.csv"

with open(txt_file, "r") as in_text:
    in_reader = csv.reader(in_text, delimiter = '\t')
    with open(csv_file, "w") as out_csv:
        out_writer = csv.writer(out_csv)
        for row in in_reader:
            out_writer.writerow(row)
            
txt_file = r"plasmid_blast/plasmidinf.txt"
csv_file = r"plasmid_blast/plasmidinf.csv"

with open(txt_file, "r") as in_text:
    in_reader = csv.reader(in_text, delimiter = '\t')
    with open(csv_file, "w") as out_csv:
        out_writer = csv.writer(out_csv)
        for row in in_reader:
            out_writer.writerow(row)

Retrieving gap data from NCBI - something that might have to be done manually

I'll probably create an independently formatted table then incorporate that into the main one somehow

In [340]:
import os

def singleretrieve(infile):
    
    single = open(infile)
    
    outtab = []
    #    outtab formatting - 
    #file name
    #header name (line 1)
    #sample name (line 2)
    #organism (line 3)
    #attributes - sub list or 2 pair lists (line 4 onwards)
    #description  (after attributes)
    #(skip ketwords if present) biosample accession
    #"ID" - same line as biosample
    
    #file name
    outtab.append(infile)
    
    #header name
    line = single.readline()
    line = line[3:-1]
    outtab.append(line)
    
    #sample name
    line = single.readline()
    key1 = "Sample name: "
    pos = line.find(key1) + len(key1)
    sample = ""
    while(line[pos] != '\n' and line[pos] != ';'):
        sample += line[pos]
        pos += 1
    outtab.append(sample)
    
    #organism
    line = single.readline()
    key2 = "Organism: "
    line = line[(line.find(key2) + len(key2)):-1]
    outtab.append(line)
    
    #attributes
    line = single.readline()
    line = single.readline()
    fullatt = []
    while(line[4] == "/"):
        curatt = []
        curlabel = ""
        pos = 5
        while(line[pos] != '='):
            curlabel += line[pos]
            pos += 1
        curatt.append(curlabel)
        pos += 2
        curentry = ""
        while(line[pos] != '\"'):
            curentry += line[pos]
            pos += 1
        curatt.append(curentry)
        fullatt.append(curatt)
        line = single.readline()
    outtab.append(fullatt)
    
    #description
    if(line.find("Description") != -1):
        line = single.readline()
        outtab.append(line[:-1])
    else:
        outtab.append("No description")
    
    #biosample and ID
    if(outtab[-1] != "No description"):
        line = single.readline()
    key3 = "Accession: "
    key4 = "ID: "
    while(line.find(key3) == -1):
        line = single.readline()
    biosamp = line[(line.find(key3) + len(key3)):(line.find(key4) - 1)]
    theid = line[(line.find(key4) + len(key4)):(len(line) - 1)]
    outtab.append(biosamp)
    outtab.append(theid)
    
    return(outtab)
    

In [341]:
singleheader = ["File name","Header name","Sample name","Organism","Attributes","Description","Biosample acession","ID"]

In [342]:
#all 13 NCBI files in 1 list

def fullretrieve(directory):
    
    fulllist = []
    
    for subdir, dirs, files in os.walk(directory):
        for i in range(len(files)):
            filename = directory + "/" + files[i]
            print(filename)
            curlist = singleretrieve(filename)
            fulllist.append(curlist)
    
    return(fulllist)
            

In [343]:
fullgaplist = fullretrieve("gapinf")

gapinf/biosample_result(9).txt
gapinf/biosample_result(11).txt
gapinf/biosample_result(7).txt
gapinf/biosample_result(4).txt
gapinf/biosample_result(3).txt
gapinf/biosample_result(2).txt
gapinf/biosample_result(6).txt
gapinf/biosample_result(12).txt
gapinf/biosample_result(10).txt
gapinf/biosample_result.txt
gapinf/biosample_result(1).txt
gapinf/biosample_result(8).txt
gapinf/biosample_result(5).txt


In [346]:
allattributes = []
attributecount = []
for i in range(len(fullgaplist)):
    for j in range(len(fullgaplist[i][4])):
        pos = -1
        for k in range(len(allattributes)):
            if(allattributes[k] == fullgaplist[i][4][j][0]):
                pos = k
                continue
        if(pos == -1):
            allattributes.append(fullgaplist[i][4][j][0])
            attributecount.append(1)
        else:
            attributecount[pos] += 1

In [452]:
for i in range(len(allattributes)):
    curprint = allattributes[i]
    for j in range(35 - len(allattributes[i])):
        curprint += " "
    curprint += "\t"
    curprint += str(attributecount[i])
    print(curprint)

strain                             	13
collection date                    	13
geographic location                	13
host                               	11
broad-scale environmental context  	9
latitude and longitude             	9
sample type                        	7
isolation and growth condition     	6
local-scale environmental context  	6
number of replicons                	6
reference for biomaterial          	6
environmental medium               	6
temperature                        	5
biomaterial provider               	4
collected by                       	4
culture collection                 	4
isolation source                   	4
mating type                        	3
host tissue sampled                	3
serotype                           	3
sample size                        	3
depth                              	3
passage history                    	3
altitude                           	3
genotype                           	2
identified by                      	2
lab host

Polish/gather components to create usable table for csv conversion

In [355]:
#find original species tags/numbers to compare easily to other 2 main tables

#batchgap.txt is in same order as file downloads

import os

batchorder = []
batchgap = open("batchgap.txt")

line = batchgap.readline()
while(line):
    if(line[-1:] == '\n'):
        line = line [:-1]
    batchorder.append([line,-1])
    line = batchgap.readline()
    
orginf = open("csvs/organisminf.txt")

line = orginf.readline()
while(line):
    curnum = ""
    curname = ""
    curpos = 0
    while(line[curpos] != "\t"):
        curnum += line[curpos]
        curpos += 1
    curpos += 1
    while(line[curpos] != '\t'):
        curname += line[curpos]
        curpos += 1
    for i in range(len(batchorder)):
        if(batchorder[i][0] == curname):
            batchorder[i][1] = curnum
    line = orginf.readline()
    

In [360]:
batchorder[0]

['Rhizobium etli strain NXC12', '0']

In [365]:
#add original names and numbers to fullgaplist using batchorder

singleheader = ["File name","Header name","Sample name","Organism","Attributes","Description","Biosample acession","ID","ognum","ogname"]

for i in range(len(fullgaplist)):
    curfile = fullgaplist[i][0]
    curnum = ""
    pos = curfile.find("(") + 1
    if(pos == 0):
        curnum = "0"
    else:
        while(curfile[pos] != ")"):
            curnum += curfile[pos]
            pos += 1
    
    curnum = int(curnum)
    fullgaplist[i].append(batchorder[curnum][1])
    fullgaplist[i].append(batchorder[curnum][0])

In [367]:
fullgaplist[0]

['gapinf/biosample_result(9).txt',
 'MIGS Cultured Bacterial/Archaeal sample from Rhizobium sp. CCGE531',
 'CCGE531',
 'Rhizobium sp. CCGE531',
 [['strain', 'CCGE531'],
  ['collection date', 'June-2016'],
  ['broad-scale environmental context', 'terrestrial biome'],
  ['local-scale environmental context', 'soil'],
  ['environmental medium', 'agricultural soil'],
  ['geographic location', 'Mexico: Veracruz, Ayahualulco, Ocotepec'],
  ['host', 'Phaseolus'],
  ['isolation and growth condition', 'PY at 28 degrees'],
  ['latitude and longitude', '19 N 97 W'],
  ['number of replicons', '5'],
  ['reference for biomaterial',
   'Rhizobium etli, R. leguminosarum and a new Rhizobium genospecies from Phaseolus dumosus']],
 'Rhizobium etli, R. leguminosarum and a new Rhizobium genospecies from Phaseolus dumosus',
 'SAMN10118218',
 '10118218',
 '42',
 'Rhizobium sp. CCGE531']

In [368]:
#manually enter relevent attributes from description, make copy of fullgaplist to revert if mistakes are made

import numpy as np
fgl2 = np.copy(fullgaplist)
fgl2 = np.ndarray.tolist(fgl2)

In [410]:
cur = fullgaplist[11]
print(cur[len(cur)-1])
print(" ")
print(cur[5])
print(" ")
maxlen = 0
for i in range(len(cur[4])):
    if(len(cur[4][i][0]) > maxlen):
        maxlen = len(cur[4][i][0])
maxlen += 2
for i in range(len(cur[4])):
    curprint = cur[4][i][0]
    for j in range(maxlen - len(cur[4][i][0])):
        curprint += " "
    curprint += cur[4][i][1]
    print(curprint)

Rhizobium pusense strain CFBP5875
 
Academia Sinica, Chih-Horng Kuo
 
strain                             CFBP5875
collection date                    NA
broad-scale environmental context  NA
local-scale environmental context  NA
environmental medium               NA
geographic location                NA
isolation and growth condition     NA
latitude and longitude             NA
number of replicons                NA
reference for biomaterial          NA


In [413]:
fgl2[4][4][1][1] = "red clover, Trifolium praetense"
fgl2[4][4][3][1] = "USA: Arlington Farms, VA"
fgl2[11][4][5][1] = "Taiwan"
fgl2[12][4][2][1] = "pea plant, Pisum sativum"

print(fgl2[12][4][2])

['host', 'pea plant, Pisum sativum']


---manual description entry doc--

Rhizobium sp. CCGE531 - no additional information

Rhizobium sp. N4311 - no additional information

Rhizobium jaguaris strain CCGE525 - no additional information

Rhizobium leguminosarum bv. viciae strain BIHB 1148 - no additional information

Rhizobium leguminosarum strain ATCC 14479 - "Rhizobium leguminosarum ATCC 14479 was obtained from the American Type Culture Collection. It was isolated at Arlington Farms, VA and is most effective on the red clover, Trifolium praetense." 
 - geographic location changed from "Not applicable" to "USA: Arlington Farms, VA"
 - host changed from "Not applicable" to "red clover, Trifolium praetense"

Rhizobium leguminosarum strain Norway - no discription was available

Rhizobium leguminosarum bv. viciae strain UPM791 - "This endosymbiotic bacteria belongs to the biovar viciae, able to nodulate legumes in the Tribe Viciae (Pisum, Lens, Vicia, Lathyrus, ). This strain is streptomycin-resistant. The host-specificity and nitrogen fixation genes are carried on large plasmids (of which it has 5)." is useful information but nothing will be added to attributes

Rhizobium sp. TAL182 - no additional information

Rhizobium sp. CCGE532 - no additional information

Rhizobium etli strain NXC12 - no additional information

Rhizobium leguminosarum strain Vaf-108 - no additional information

Rhizobium pusense strain CFBP5875 - "Academia Sinica, Chih-Horng Kuo" this was manually added by me as the name was available on the NCBI page somewhere. This person/Academia Sinica is located in Taiwan so that will be assumed as the location
 - geographic location changed from "NA" to "Taiwan"
 
Rhizobium leguminosarum bv. viciae strain BIHB 1217 - "Plant growth promoting nitrogen-fixing microbial inoculant for pea"
 - host change from "not applicable" to "pea plant, Pisum sativum"


In [448]:
#sort allattributes from highest to lowest data count using attributecount

def attsort(allatt,attcount):
    cont = True
    while(cont):
        cont = False
        curlow = attcount[0]
        moveind = -1
        for i in range(len(attcount)):
            if(attcount[i] > curlow):
                moveind = i
                cont = True
                break
            if(attcount[i] < curlow):
                curlow = attcount[i]
                
        if(moveind != -1):
            swapind = -1
            for j in range(len(attcount)):
                if(attcount[moveind] > attcount[j]):
                    swapind = j
                    break
            if(swapind == -1):
                print("You broke it")
            tempall = allatt[moveind]
            allatt[moveind] = allatt[swapind]
            allatt[swapind] = tempall
            tempcount = attcount[moveind]
            attcount[moveind] = attcount[swapind]
            attcount[swapind] = tempcount

In [451]:
attsort(allattributes,attributecount)

In [458]:
#apply it to fgl2 to be in numerical organism number

import numpy as np
flg3 = np.copy(fgl2)
flg3 = np.ndarray.tolist(flg3)

flgorder = []
for i in range(len(flg3)):
    flgorder.append(int(flg3[i][8]))

In [466]:
attsort(flg3,flgorder)
flg3.reverse()

Create table for csv conversion using above components

Organization should be
 - My assigned organism number (8)
 - #Organism/name used on other 2 tables (9)
 - Symbiotic plasmid (taken from organisminf table, will need to import
 - header name (1)
 - sample name (2)
 - organism (3)
 - biosample accession (6)
 - ID (7)
 - The 38 now sort attributes (4) - will need to add these differently
 

In [478]:
for i in range(len(singleheader)):
    print(singleheader[i])

File name
Header name
Sample name
Organism
Attributes
Description
Biosample acession
ID
ognum
ogname


In [475]:
gapheader = ["Organism number","#Organism/Name","Symbiotic Plasmid","Header name","Sample name","Organism","Biosample","ID"]
for i in range(len(allattributes)):
    gapheader.append(allattributes[i])

In [483]:
gapinf = []
gapinf.append(gapheader)

for i in range(len(flg3)):
    currow = []
    currow.append(flg3[i][8])
    currow.append(flg3[i][9])
    currow.append('') #fill in later with organisminf Symbiotic plasmid column
    currow.append(flg3[i][1])
    currow.append(flg3[i][2])
    currow.append(flg3[i][3])
    currow.append(flg3[i][6])
    currow.append(flg3[i][7])
    for j in range(8,len(gapheader)):
        curatt = gapheader[j]
        attmatch = ''
        for j in range(len(flg3[i][4])):
            if(flg3[i][4][j][0] == curatt):
                attmatch = flg3[i][4][j][1]
        currow.append(attmatch)
    gapinf.append(currow)

In [488]:
for i in range(1,len(gapinf)):
    curnum = gapinf[i][0]
    for j in range(1,len(organisminf)):
        if(curnum == organisminf[j][0]):
            gapinf[i][2] = organisminf[j][2]

In [491]:
#LIST TO TEXT

import os

funcplas = open("csvs/gapinf.txt",'w')

for i in range(len(gapinf)):
    for j in range(len(gapinf[i])):
        funcplas.write(str(gapinf[i][j]))
        if(j < (len(gapinf[i]) - 1)):
            funcplas.write('\t')
    funcplas.write('\n')

In [492]:
#TEXT TO CSV 

import csv

txt_file = r"csvs/gapinf.txt"
csv_file = r"csvs/gapinf.csv"

with open(txt_file, "r") as in_text:
    in_reader = csv.reader(in_text, delimiter = '\t')
    with open(csv_file, "w") as out_csv:
        out_writer = csv.writer(out_csv)
        for row in in_reader:
            out_writer.writerow(row)